# TP2 - Détection d'Anomalies (Auto-Encodeurs)

Ce notebook charge les données ECG préparées depuis `data/ECG/` et compare deux architectures d'auto-encodeurs (AE) :
  1.  Un AE classique (simple).
  2.  Un AE débruiteur (Denoising AE).

 L'entraînement se fait *uniquement* sur les données normales (`X_train`), la recherche de seuil sur les données de validation (`X_val`), et l'évaluation finale sur les données de test (`X_test`).

In [ ]:
# --- Imports ---
from pathlib import Path
import sys
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report
)

In [2]:
# --- Configuration des Chemins ---

PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED_PATH = PROJECT_ROOT / "data" / "ECG"
PROJECT_FCT = PROJECT_ROOT / "src"
sys.path.append(str(PROJECT_FCT))

print(f"Chemin du projet racine : {PROJECT_ROOT}")
print(f"Chargement des données depuis : {DATA_PROCESSED_PATH}")

Chemin du projet racine : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git
Chargement des données depuis : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git\data\ECG


In [3]:
# --- Chargement des modules locaux ---
# (Utilise la structure de /src comme PreparationData.ipynb)
try:
    # Supposons que ces classes/fonctions existent dans src/
    from models import Autoencoder, DenoisingAutoencoder 
    from train import train_autoencoder, train_denoising_autoencoder
    from utils import evaluate_model_threshold
    print("Modules locaux 'models', 'train' et 'utils' chargés avec succès.")
except ImportError:
    print("Attention: Les fichiers .py de /src n'ont pas été trouvés.")
    print("Définition de classes/fonctions de remplacement (placeholders).")

c:\Users\sebdr\anaconda3\envs\udes_sd_a25\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Modules locaux 'models', 'train' et 'utils' chargés avec succès.


In [4]:
# --- Chargement des données ---

print("Chargement des données ECG préparées (depuis data/ECG/)...")
try:
    X_train_scaled = np.load(DATA_PROCESSED_PATH / 'ecg_X_train_scaled.npy')
    X_val_scaled = np.load(DATA_PROCESSED_PATH / 'ecg_X_val_scaled.npy')
    X_test_scaled = np.load(DATA_PROCESSED_PATH / 'ecg_X_test_scaled.npy')
    y_train = np.load(DATA_PROCESSED_PATH / 'ecg_y_train.npy')
    y_val = np.load(DATA_PROCESSED_PATH / 'ecg_y_val.npy')
    y_test = np.load(DATA_PROCESSED_PATH / 'ecg_y_test.npy')
except FileNotFoundError:
    print(f"ERREUR: Fichiers non trouvés dans {DATA_PROCESSED_PATH}")
    print("Veuillez d'abord exécuter le notebook 'PreparationData.ipynb'.")

Chargement des données ECG préparées (depuis data/ECG/)...


In [5]:
# --- Conversion en Tensors PyTorch ---
BATCH_SIZE = 64
X_train_t = torch.tensor(X_train_scaled.astype(np.float32))
X_val_t = torch.tensor(X_val_scaled.astype(np.float32))
X_test_t = torch.tensor(X_test_scaled.astype(np.float32))
y_val_t = torch.tensor(y_val.astype(np.float32))
y_test_t = torch.tensor(y_test.astype(np.float32))

# Dataset (Train ne contient que des données normales)
train_dataset = TensorDataset(X_train_t, torch.tensor(y_train.astype(np.float32)))
val_dataset = TensorDataset(X_val_t, y_val_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("DataLoaders PyTorch créés.")
input_dim = X_train_scaled.shape[1]

DataLoaders PyTorch créés.


# 1. Auto-encodeur (AE) Classique

In [6]:
# --- Entraînement AE ---
model_ae = Autoencoder(input_dim)
model_ae = train_autoencoder(model_ae, train_loader, n_epochs=50, lr=1e-3)

Début de l'entraînement (AE) pour 50 époques...
Époque [10/50], Perte (Loss): 0.299233
Époque [20/50], Perte (Loss): 0.190413
Époque [30/50], Perte (Loss): 0.149218
Époque [40/50], Perte (Loss): 0.116133
Époque [50/50], Perte (Loss): 0.099761
Entraînement terminé en 1.97 secondes.


In [7]:
# --- Seuil AE (sur Validation) ---
threshold_ae, f1_val_ae = evaluate_model_threshold(model_ae, val_loader, y_val)
print(f"Seuil optimal (AE) trouvé: {threshold_ae:.6f} (F1-Score Val: {f1_val_ae:.4f})")

Calcul des erreurs et du seuil optimal...
Seuil optimal (AE) trouvé: 0.189786 (F1-Score Val: 0.9881)


C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git\src\utils.py:217: RuntimeWarning: invalid value encountered in divide
  f1_scores = (2 * precisions * recalls) / (precisions + recalls)


In [8]:
# --- Évaluation AE (sur Test) ---
print("Évaluation de l'AE Classique sur l'ensemble de test...")
model_ae.eval()
test_errors_ae = []
with torch.no_grad():
    for inputs, _ in test_loader:
        recon = model_ae(inputs)
        errors = torch.mean((inputs - recon) ** 2, dim=1)
        test_errors_ae.extend(errors.numpy())

y_pred_ae = (np.array(test_errors_ae) > threshold_ae).astype(int) # 1 = Anomalie
y_test_np = y_test # Labels réels

# --- Métriques de performance (AE) ---
accuracy_ae = accuracy_score(y_test_np, y_pred_ae)
precision_ae = precision_score(y_test_np, y_pred_ae, pos_label=1)
recall_ae = recall_score(y_test_np, y_pred_ae, pos_label=1)
f1_ae = f1_score(y_test_np, y_pred_ae, pos_label=1)
roc_auc_ae = roc_auc_score(y_test_np, test_errors_ae)

print(f"Accuracy (AE): {accuracy_ae:.4f}")
print(f"Precision (AE): {precision_ae:.4f}")
print(f"Recall (AE): {recall_ae:.4f}")
print(f"F1-Score (AE): {f1_ae:.4f}")
print(f"ROC-AUC (AE): {roc_auc_ae:.4f}")
print("\nRapport de Classification (AE) sur Test:")
print(classification_report(y_test_np, y_pred_ae, target_names=['Normal (0)', 'Anomalie (1)']))

Évaluation de l'AE Classique sur l'ensemble de test...
Accuracy (AE): 0.9733
Precision (AE): 0.9720
Recall (AE): 0.9949
F1-Score (AE): 0.9833
ROC-AUC (AE): 0.9448

Rapport de Classification (AE) sur Test:
              precision    recall  f1-score   support

  Normal (0)       0.98      0.89      0.93       624
Anomalie (1)       0.97      0.99      0.98      2336

    accuracy                           0.97      2960
   macro avg       0.98      0.94      0.96      2960
weighted avg       0.97      0.97      0.97      2960



# 2. Auto-encodeur Débruiteur (DAE)

In [9]:
# --- Entraînement DAE ---
model_dae = DenoisingAutoencoder(input_dim, noise_factor=0.2)
model_dae = train_denoising_autoencoder(model_dae, train_loader, n_epochs=50, lr=1e-3, noise_factor=0.2)

Début de l'entraînement (DAE) pour 50 époques...
Époque [10/50], Perte (Loss): 0.316329
Époque [20/50], Perte (Loss): 0.189910
Époque [30/50], Perte (Loss): 0.155188
Époque [40/50], Perte (Loss): 0.136216
Époque [50/50], Perte (Loss): 0.111829
Entraînement DAE terminé en 2.06 secondes.


In [10]:
# --- Seuil DAE (sur Validation) ---
threshold_dae, f1_val_dae = evaluate_model_threshold(model_dae, val_loader, y_val)
print(f"Seuil optimal (DAE) trouvé: {threshold_dae:.6f} (F1-Score Val: {f1_val_dae:.4f})")

Calcul des erreurs et du seuil optimal...
Seuil optimal (DAE) trouvé: 0.190498 (F1-Score Val: 0.9838)


C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git\src\utils.py:217: RuntimeWarning: invalid value encountered in divide
  f1_scores = (2 * precisions * recalls) / (precisions + recalls)


In [12]:
# --- Évaluation DAE (sur Test) ---
print("Évaluation du DAE sur l'ensemble de test...")
model_dae.eval()
test_errors_dae = []
with torch.no_grad():
    for inputs, _ in test_loader:
        recon = model_dae(inputs)
        errors = torch.mean((inputs - recon) ** 2, dim=1)
        test_errors_dae.extend(errors.numpy())

y_pred_dae = (np.array(test_errors_dae) > threshold_dae).astype(int)

# --- Métriques de performance (DAE) ---
accuracy_dae = accuracy_score(y_test_np, y_pred_dae)
precision_dae = precision_score(y_test_np, y_pred_dae, pos_label=1)
recall_dae = recall_score(y_test_np, y_pred_dae, pos_label=1)
f1_dae = f1_score(y_test_np, y_pred_dae, pos_label=1)
roc_auc_dae = roc_auc_score(y_test_np, test_errors_dae)

print(f"Accuracy (DAE): {accuracy_dae:.4f}")
print(f"Precision (DAE): {precision_dae:.4f}")
print(f"Recall (DAE): {recall_dae:.4f}")
print(f"F1-Score (DAE): {f1_dae:.4f}")
print(f"ROC-AUC (DAE): {roc_auc_dae:.4f}")
print("\nRapport de Classification (DAE) sur Test:")
print(classification_report(y_test_np, y_pred_dae, target_names=['Normal (0)', 'Anomalie (1)']))

Évaluation du DAE sur l'ensemble de test...
Accuracy (DAE): 0.9676
Precision (DAE): 0.9718
Recall (DAE): 0.9876
F1-Score (DAE): 0.9796
ROC-AUC (DAE): 0.9430

Rapport de Classification (DAE) sur Test:
              precision    recall  f1-score   support

  Normal (0)       0.95      0.89      0.92       624
Anomalie (1)       0.97      0.99      0.98      2336

    accuracy                           0.97      2960
   macro avg       0.96      0.94      0.95      2960
weighted avg       0.97      0.97      0.97      2960



# 3. Comparaison AE vs DAE (et IF)

In [13]:
# --- Résultats Isolation Forest reportés---
accuracy_if = 0.9622
precision_if = 0.9572
recall_if = 0.9966
f1_if = 0.9765
roc_auc_if = 0.9218

In [15]:
data_summary = {
     "Isolation Forest": {
         "Accuracy": accuracy_if, "Precision (Anomalie)": precision_if, 
         "Recall (Anomalie)": recall_if, "F1-Score (Anomalie)": f1_if, "ROC-AUC": roc_auc_if
     },
     "Auto-encodeur (AE)": {
         "Accuracy": accuracy_ae, "Precision (Anomalie)": precision_ae, 
         "Recall (Anomalie)": recall_ae, "F1-Score (Anomalie)": f1_ae, "ROC-AUC": roc_auc_ae
     },
     "Denoising AE (DAE)": {
         "Accuracy": accuracy_dae, "Precision (Anomalie)": precision_dae, 
         "Recall (Anomalie)": recall_dae, "F1-Score (Anomalie)": f1_dae, "ROC-AUC": roc_auc_dae
     }
 }
 
df_anomaly_results = pd.DataFrame(data_summary).T
df_anomaly_results_styled = df_anomaly_results.style.format("{:.4f}").highlight_max(axis=0, color='green')
print("--- Tableau de Comparaison des Modèles de Détection d'Anomalies (sur X_test) ---")
df_anomaly_results_styled

--- Tableau de Comparaison des Modèles de Détection d'Anomalies (sur X_test) ---


,Accuracy,Precision (Anomalie),Recall (Anomalie),F1-Score (Anomalie),ROC-AUC
Isolation Forest,0.9622,0.9572,0.9966,0.9765,0.9218
Auto-encodeur (AE),0.9733,0.9720,0.9949,0.9833,0.9448
Denoising AE (DAE),0.9676,0.9718,0.9876,0.9796,0.9430
